<img style="float: right;" src="files/EHT.jpg" width="100" height="100">

## Mejorando la red neuronal con hiperparámetros con Keras

En este último tutorial se seguirá utilizando el mismo conjunto de datos, y la misma librería. En este caso se hará un código para poder hacer análsis de hiperparámetros en rangos definidos.

Utilizaremos la función [GridSearchCV](https://scikit-learn.org/stable/modules/grid_search.html) para generar una red de parámetros que se van a probar. 

Hacemos lo usual, importar los datos y separarlos en entradas y salida.

In [1]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier

dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")

X = dataset[:,0:-1]
y = dataset[:,-1]

Using TensorFlow backend.
F:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529

Usamos una función para poder asignar valores con ayuda de los parámetros de entrada. En esta ocasión solo pondremos como variable de entrada el número de neuronas en la primera capa oculta (neurons), de la segunda capa (neurons2) y el optimizador (optimizer).

In [2]:
def create_model(neurons=1, neurons2=1, optimizer='adam'):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X.shape[1], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(neurons2, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

Ahora generamos el clasificador con la arquitectura definida en la función create_model

In [3]:
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=10)

Aquí hacemos un lista para cada parámetro con los rangos que se van a probar. GridSerachCV necesita un [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) con las listas y el parámetro al que pertenecen.

In [4]:
neurons = list(range(10, 15, 1))
neurons2 = list(range(10, 15, 1))
optimizer = ['SGD', 'RMSprop', 'Adadelta', 'Adam']

param_grid = dict(neurons=neurons, neurons2=neurons2, optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

Después se hace en el entrenamiento de grid usando el método fit

In [5]:
grid_result = grid.fit(X, y)

F:\Users\USER\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch

Una vez entrenada la red se hace imprimen los resultados (promedios y desviaciones).

In [6]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    

0.669383 (0.055406) with: {'neurons': 10, 'neurons2': 10, 'optimizer': 'SGD'}
0.674612 (0.069966) with: {'neurons': 10, 'neurons2': 10, 'optimizer': 'RMSprop'}
0.691529 (0.056621) with: {'neurons': 10, 'neurons2': 10, 'optimizer': 'Adadelta'}
0.658985 (0.061191) with: {'neurons': 10, 'neurons2': 10, 'optimizer': 'Adam'}
0.665495 (0.062134) with: {'neurons': 10, 'neurons2': 11, 'optimizer': 'SGD'}
0.679756 (0.041846) with: {'neurons': 10, 'neurons2': 11, 'optimizer': 'RMSprop'}
0.678499 (0.050558) with: {'neurons': 10, 'neurons2': 11, 'optimizer': 'Adadelta'}
0.675919 (0.062808) with: {'neurons': 10, 'neurons2': 11, 'optimizer': 'Adam'}
0.682387 (0.042049) with: {'neurons': 10, 'neurons2': 12, 'optimizer': 'SGD'}
0.711009 (0.052421) with: {'neurons': 10, 'neurons2': 12, 'optimizer': 'RMSprop'}
0.673287 (0.047251) with: {'neurons': 10, 'neurons2': 12, 'optimizer': 'Adadelta'}
0.684976 (0.057834) with: {'neurons': 10, 'neurons2': 12, 'optimizer': 'Adam'}
0.649860 (0.052554) with: {'neuron

Imprimimos los mejores resultados de entre todos los modelos.
Es importante destacar que se debe hacer una busqueda m

In [7]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.711009 using {'neurons': 10, 'neurons2': 12, 'optimizer': 'RMSprop'}


Todos los parámetros se pueden analizar, en [esta página](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/) hay ejemplos haciendo análsis de varios parámetros.